In [ ]:
import numpy as np
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session
from qiskit.visualization import plot_histogram
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer

my_token = "17d69979f69d8a1c1c3577d0416e88526a1ae3592b880bc2d13c58463880cfba102a7ccdbe99cdbd62c8a9e1546897973fa6a73d00e43455edb7fb0cb8fb0e9b"

def create_superposition(qc, qubits):
    """Applies Hadamard gates to place qubits into superposition."""
    for q in qubits:
        qc.h(q)

def oracle(qc):
    """Implements the oracle for marking the |11> state with a phase flip."""
    qc.cz(0, 1)  # Phase flip when both qubits are |1>
    qc.barrier()

def diffuser(qc):
    """Implements the quantum diffusion (amplitude amplification)."""
    qc.h([0, 1])  # Step 1: Apply Hadamard
    qc.x([0, 1])  # Step 2: Apply X-gates (inversion about zero)
    qc.cz(0, 1)   # Step 3: Controlled-Z (inversion about mean)
    qc.x([0, 1])  # Step 4: Apply X-gates again
    qc.h([0, 1])  # Step 5: Apply Hadamard again
    qc.barrier()

def grover_circuit():
    """Constructs the complete Grover search circuit."""
    qc = QuantumCircuit(2, 2)

    # Step 1: Initialize in Superposition
    create_superposition(qc, [0, 1])

    # Step 2: Apply Oracle
    oracle(qc)

    # Step 3: Apply Diffuser
    diffuser(qc)

    # Step 4: Measure the qubits
    qc.measure([0, 1], [0, 1])

    return qc

def run_on_ibm(qc):
    """Runs the circuit on IBM Quantum Experience using Sampler."""
    service = QiskitRuntimeService(channel="ibm_quantum", token=my_token)
    backend = service.backend("ibm_brisbane")  # Ensure the backend is available

    # Transpile the circuit for the backend
    transpiled_qc = transpile(qc, backend)

    # Use IBM Quantum session
    with Session(backend=backend):
        sampler = Sampler()
        job = sampler.run([transpiled_qc])  # Pass as a list
        result = job.result()

    # Print the result object
    print("Result object:", result)
    print("Available attributes:", dir(result))



# Generate and visualize the quantum circuit
qc = grover_circuit()
print(qc.draw())  # Print quantum circuit diagram

# Run on IBM Quantum Simulator
run_on_ibm(qc)


     ┌───┐    ░ ┌───┐┌───┐   ┌───┐┌───┐ ░ ┌─┐   
q_0: ┤ H ├─■──░─┤ H ├┤ X ├─■─┤ X ├┤ H ├─░─┤M├───
     ├───┤ │  ░ ├───┤├───┤ │ ├───┤├───┤ ░ └╥┘┌─┐
q_1: ┤ H ├─■──░─┤ H ├┤ X ├─■─┤ X ├┤ H ├─░──╫─┤M├
     └───┘    ░ └───┘└───┘   └───┘└───┘ ░  ║ └╥┘
c: 2/══════════════════════════════════════╩══╩═
                                           0  1 


In [ ]:
# 17d69979f69d8a1c1c3577d0416e88526a1ae3592b880bc2d13c58463880cfba102a7ccdbe99cdbd62c8a9e1546897973fa6a73d00e43455edb7fb0cb8fb0e9b

In [7]:
# Authenticate
service = QiskitRuntimeService(channel="ibm_quantum", token=my_token)
# Print available backends
print(service.backends())

[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_kyiv')>, <IBMBackend('ibm_sherbrooke')>]
